<a href="https://colab.research.google.com/github/OxygenEnthusiast/docker-action-hello-world/blob/main/gen_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In your google drive, make sure to add the corresponding project directory containing the subdirectory "images" found in the [omi-repo](https://github.com/jcpeterson/omi) as well as lhe `attribute_means.csv` file from the same repository. If already calculated add the projected latent vectors and/or even better the `attr_dirs.pkl` file.

In [1]:
# @title Path definitions { display-mode: "form" }

PROJECT_DIR = "gen_faces" #@param {type:"string"}
PROJECT_PATH = f"/content/drive/MyDrive/{PROJECT_DIR}"
SAVE_IMAGE_IN_DRIVE = True #@param {type:"boolean"}
SAVE_IMAGE_PATH = f"{PROJECT_PATH}/generated_images/" if SAVE_IMAGE_IN_DRIVE else "/content"

image_directory = f"{PROJECT_PATH}/images"
vec_directory = f"{PROJECT_PATH}/l_vecs"

import pathlib
for path in [PROJECT_PATH, image_directory, vec_directory]:
  pathlib.Path(path).mkdir(exist_ok=True)

# Prerequisites

## Install Stylegan for face generation

In [2]:
import sys
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja
sys.path.insert(0, "/content/stylegan2-ada-pytorch")
!wget "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 7.92 MiB/s, done.
Resolving deltas: 100% (57/57), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 3.4 MB/s eta 0:00:00
--2023-08-28 08:40:47--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 52.84.18.96, 52.84.18.33, 52.84.18.74, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|52.84.18.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381624121 (364M) [binary/octet-stream]
Saving to: ‘ffhq.pkl’

ffhq.pkl            100%[===================>] 363.94M  30.4MB/s    in 13s     

2023-08-28 08:41:01 (28.7 MB/s) - ‘ffhq.pkl’ saved [381624121/381624121]



## Project Images onto Facespace

First we use the stylegan projector tool to get the corresponding latent vector in w-space for each image used in the [omi-dataset](https://github.com/jcpeterson/omi). This can take up to 2 days so be sure to save the results in the corresponding directory. Only images where no corresponing latent vector is found will be projected.

In [3]:
import os
from tqdm.notebook import tqdm


NETWORK = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl"

def remove_extension(file):
  if file.lower().endswith('.jpg'):
    return os.path.splitext(file)[0]

def get_imagenames():
  original = [remove_extension(f) for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]
  done =  os.listdir(vec_directory)
  res = [f for f in original if f not in done]
  print(f"original: {len(original)}, done: {len(done)}, left: {len(res)}")
  return res

def gen_cmd(imagename):
  return  f"python stylegan2-ada-pytorch/projector.py\
        --save-video 0 --num-steps 1000\
        --outdir={vec_directory}/{imagename}\
        --target={image_directory}/{imagename}.jpg\
        --network={NETWORK}"

def project_images():
  for imagename in tqdm(get_imagenames()):
    !{gen_cmd(imagename)}


## Calculate attribute directions

We now load the ratings from the [omi-dataset](https://github.com/jcpeterson/omi) and match them to the corresponding latent vecors. Then we do linear regression on each attribute to get the linear combination which characterizes it most. We call these combinations attribute directions.

In [4]:
import numpy as np
import os
from sklearn.linear_model import RidgeCV
from tqdm.notebook import tqdm
import pandas as pd
import pickle


def load_w_space_vecs():
  return {int(d):np.load(f"{vec_directory}/{d}/projected_w.npz")['w'] for d in tqdm(os.listdir(vec_directory))}

def load_ratings():
  ratings = pd.read_csv(f"{PROJECT_PATH}/attribute_means.csv")
  ratings.set_index('stimulus',inplace=True)
  return ratings

def prepare_vecs_for_regression(vecs):
  return np.array([value.flatten() for _, value in sorted(vecs.items())])

def find_attr_dir(specific_ratings_):
  X, y = prepare_vecs_for_regression(load_w_space_vecs()), specific_ratings_.to_numpy()
  clf = RidgeCV().fit(X, y)
  return clf.coef_.reshape(X[1].shape)

def find_all_attr_dirs():
  ratings = load_ratings()
  return {ratingname : find_attr_dir(ratings[ratingname]) for ratingname in ratings.columns}

def load_attr_dirs(path):
  with open(path, "rb") as f:
    return pickle.load(f)

def save_attr_dirs(path):
  with open(path, "wb") as f:
      pickle.dump(attr_dirs, f)

def get_attr_dirs():
  path = f"{PROJECT_PATH}/attr_dirs.pkl"
  if os.path.exists(path):
    return load_attr_dirs(path)
  project_images()
  attr_dirs = find_all_attr_dirs()
  save_attr_dirs(path)
  return attr_dirs

In [5]:
r_attr_dirs = get_attr_dirs()

In [6]:
import numpy as np
def normalize_dir(dir):
  return dir / np.linalg.norm(dir)
attr_dirs = {attr:normalize_dir(val) for attr, val in r_attr_dirs.items()}

## random latent vector generation


For reference on mapping parameters see [here](https://github.com/NVlabs/stylegan/tree/1e0d5c781384ef12b50ef20a62fee5d78b38e88f#using-pre-trained-networks).

We will be working with vectors within the W+ space. To get such a vector we first generate a random z vector and then map it to W+.

In [7]:
import torch

with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()  # torch.nn.Module

def get_random_Z_space_vec():
  return torch.randn([1, G.z_dim]).cuda()

def get_random_W_space_vec():
  z, c = get_random_Z_space_vec(), None
  return G.mapping(z , c, truncation_psi=0.5, truncation_cutoff=8)


In [8]:
def get_blank_face():
  return get_random_W_space_vec().cpu().numpy()


## Figure out mean and standard deviation

We empirically calculate the mean and the covarianve matrix. Note that our sample is chosen based on a normal distribution over Z space wich is then mapped onto W. So the sample faces are not really representative of real life. A more curated sample would probably be better.

In [9]:
CALC_SIZE = 5000

def empiric_mean():
  return np.mean([get_blank_face() for _ in range(CALC_SIZE)], axis = 0)

def empiric_cov():
  return np.cov([get_blank_face().flatten() for _ in range(CALC_SIZE)], rowvar=False)

In [10]:
oa_mean = empiric_mean()
cov_mtrx = empiric_cov()

Setting up PyTorch plugin "bias_act_plugin"... Done.


Using the mean face $\mu_F\in \mathbb{R}^d$ we can calculate the mean along some direction $\mu_d\in\mathbb{R}$. To do that we use the scalar projection since the direction is normalized.
$$
\mu_d = \langle \mu_F , d \rangle
$$

In [11]:
def flat_dot(a,b):
  return np.dot(a.flatten(), b.flatten())

def mean_of_dir(dir):
  return flat_dot(oa_mean, normalize_dir(dir))

We can then calculate the standard deviation $\sigma_d$ along some direction $d$ by
$$
\sigma_d = \sqrt{ d^t\ \Sigma\ d}
$$

In [12]:
def std_of_dir(dir):
  fdir = normalize_dir(dir.flatten())
  return np.sqrt(np.matmul(np.matmul(fdir.transpose(), cov_mtrx), fdir))

## Face manipulation

Our Goal is that if for any direction $d$ we can generate a face that is average in this dimension. We call this face the base face. In a further step we want to change the face by multiples of standard deviations $\sigma_d$ in $d$. So assuming we have multiples -1,0.1 the scalar projection of the generated faces onto $d$ will be $-\sigma_d, 0, \sigma_d$ respectively.

When analysing a face along some direction we want to first standardize the face, such that the base face is average along the direction.
$$
f - ⟨f,d⟩ d  + μ_d d
=
f - (⟨f,d⟩ - μ_d) d
$$

In [13]:
def get_base_face(attr_dir):
  base = get_blank_face()
  return standardize_face(base, attr_dir, mean_of_dir(attr_dir))

def standardize_face(face, dir, mean):
  sdir = normalize_dir(dir)
  return face - sdir * (flat_dot(sdir,face)-mean)


To have meaningful step distances we want to change the magnitude of the direction $d$ to its standard deviation $σ_d$.

In [14]:
def standardize_dir(dir):
  sdir = normalize_dir(dir)
  return std_of_dir(sdir) * sdir


When editing a face we might want to edit along one attribute vector $A$ without moving along another attribute $S$. To do this we project $A$ onto $S^\perp$(the orthogonal complement of $S$). This
results in the following directiuon:
$$
A - \langle A , S \rangle S
$$


In [15]:
def orth_proj(a,b):
  assert np.isclose(np.linalg.norm(b), 1)
  return b * flat_dot(a,b)

def raw_stab_dir(a,s):
  return a - orth_proj(a,s)


## Face generation

We want to generate multiple faces, as described above. We generate them using the synthesis network. since the contrast is weird we normalize it in a very simple way.

In [16]:
import torch

def synthesize_face(l_vec):
  w = torch.from_numpy(l_vec).cuda()
  return  G.synthesis(w, noise_mode='const', force_fp32=True)

def normalize_face_contrast(image):
  min_val = float(torch.min(image))
  max_val = float(torch.max(image))
  amp = max_val - min_val
  return (image - min_val)/amp

def create_face(l_vec):
  return normalize_face_contrast(synthesize_face(l_vec))

def get_multiple_faces(base, attr_dir, mults):
  return {mult:create_face(base + attr_dir * mult) for mult in mults}


In [21]:
from torchvision.utils import save_image
from os import makedirs

def save_facematrix(face_matrix):
  for i, faces in enumerate(face_matrix):
    makedirs(f'{SAVE_IMAGE_PATH}{i}', exist_ok=True)
    for val, face in faces.items():
      save_image(face, f'{SAVE_IMAGE_PATH}{i}/{val}.png')

# Main

In [20]:
#@title Main Code { display-mode: "form" }

AMOUNT_OF_FACES = 10 #@param {type:"integer", min: 1, max:100}
ATTRIBUTE_TO_CHANGE = 'outdoors' #@param ['trustworthy', 'attractive', 'dominant', 'smart', 'age', 'gender', 'weight', 'typical', 'happy', 'familiar', 'outgoing', 'memorable', 'well-groomed', 'long-haired', 'smug', 'dorky', 'skin-color', 'hair-color', 'alert', 'cute', 'privileged', 'liberal', 'asian', 'middle-eastern', 'hispanic', 'islander', 'native', 'black', 'white', 'looks-like-you', 'gay', 'electable', 'godly', 'outdoors']
SHOULD_STABILIZE = False #@param {type:"boolean"}
STABILIZE_BY = 'age'#@param ['trustworthy', 'attractive', 'dominant', 'smart', 'age', 'gender', 'weight', 'typical', 'happy', 'familiar', 'outgoing', 'memorable', 'well-groomed', 'long-haired', 'smug', 'dorky', 'skin-color', 'hair-color', 'alert', 'cute', 'privileged', 'liberal', 'asian', 'middle-eastern', 'hispanic', 'islander', 'native', 'black', 'white', 'looks-like-you', 'gay', 'electable', 'godly', 'outdoors']
CHANGE_FACTORS = "-8,-4,0,4,8" #@param [[-10, -5, -2,  0, 2 , 5, 10], [-8, -4,  0, 4 , 8]] {allow-input: true}
CHANGE_FACTORS = [int(i) for i in CHANGE_FACTORS.split(",")]

dir = attr_dirs[ATTRIBUTE_TO_CHANGE]
if SHOULD_STABILIZE:
  dir = raw_stab_dir(dir ,attr_dirs[STABILIZE_BY])
sdir = standardize_dir(dir)
base_faces = [get_base_face(sdir) for _ in range(AMOUNT_OF_FACES)]
face_matrix = [get_multiple_faces(base_face, sdir, CHANGE_FACTORS ) for base_face in base_faces]
save_facematrix(face_matrix)